# Initial Setup

In [ ]:
!pip install pyts

In [ ]:
# Download data from Google Drive
import gdown

data_train_id = "1oG5FtbXgUgnYkQI-Om-MBq_n1s2dYUAt"
data_test_id = "1dViHrJP7Lm6nr_RhEaDB9A7IlOwy439s"
target_train_id = "1km9rjrYGDyQlLE_20Q4kLjJXWoXLgEA9"
target_test_id = "1N-xiJWDqQJq2UMaafuBwGg6ghRLvrpEM"

data_train_norm_id = "1sstp-w4HBoFiegK90uHiMwuOygNOKaEU"
data_test_norm_id = "1CyN22bU0iyYbgMwUVYNSJPc_GwqMCL1P"



gdown.download(id=data_train_id, output="data_train.npy", quiet=True)
gdown.download(id=data_test_id, output="data_test.npy", quiet=True)
gdown.download(id=target_train_id, output="target_train.npy", quiet=True)
gdown.download(id=target_test_id, output="target_test.npy", quiet=True)
gdown.download(id=data_train_norm_id, output="data_train_norm.npy", quiet=True)
gdown.download(id=data_test_norm_id, output="data_test_norm.npy", quiet=True)

In [5]:
# Load data and shuffle

import numpy as np
from sklearn.utils import shuffle
data_train = np.load("data_train.npy", allow_pickle=True)
data_test = np.load("data_test.npy")
target_train = np.load("target_train.npy")
target_test = np.load("target_test.npy")
data_train_norm = np.load("data_train_norm.npy")
data_test_norm = np.load("data_test_norm.npy")

data_train, target_train, data_train_norm = shuffle(data_train, target_train, data_train_norm, random_state=42)
data_test, target_test, data_test_norm = shuffle(data_test, target_test, data_test_norm, random_state=42)

NUM_CLASSES = 19

In [3]:
# Build models as described in the original paper

import pickle
import numpy as np
import sklearn
import sklearn.metrics
from sklearn.neighbors import KernelDensity

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

from pyts.metrics import boss, dtw

def build_mlp(input_shape,nb_classes):
    x = keras.layers.Input(shape=(input_shape))
    #a Layer instance is callable on a tensor , and returns a tensor
    x_nb = Dense(64 , activation='relu')(x)
    x_nb = keras.layers.Dropout(0.2)(x_nb)
    out = Dense(nb_classes,activation='softmax')(x_nb)
    return x, out

def build_lstm(input_shape,nb_classes):
    x = keras.layers.Input(shape=input_shape)
    x_nb = LSTM(64)(x)
    x_nb = keras.layers.Dropout(0.2)(x_nb)
    out = Dense(nb_classes,activation='softmax')(x_nb)
    return x, out

def build_fcn(input_shape,nb_classes):
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv1D(128, kernel_size=8, padding='same')(conv_x)
    # conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_x = keras.layers.Dropout(0.2)(conv_x)
    full = keras.layers.GlobalAveragePooling1D()(conv_x)
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    return x, out
def build_resnet(input_shape, n_feature_maps, nb_classes):
    #print('build conv_x')
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv1D(n_feature_maps, kernel_size=8, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    #print('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    #print('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps, kernel_size=1, padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    #print('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)

    #print('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=8, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    #print('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    #print('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=1, padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    #print('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)

    #print('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=8, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    #print('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    #print('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps * 2, kernel_size=1, padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    #print('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)

    full = keras.layers.GlobalAveragePooling1D()(y)
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    #print('        -- model was built.')
    return x, out


def build_encoder(input_shape, nb_classes):
    x = keras.layers.Input(input_shape)

    # conv block -1
    conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(x)
    conv1 = keras.layers.GroupNormalization()(conv1)
    # conv1 = tfa.layers.InstanceNormalization()(conv1)
    conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
    conv1 = keras.layers.Dropout(rate=0.2)(conv1)
    conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # conv block -2
    conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(conv1)
    conv2 = keras.layers.GroupNormalization()(conv2)
    # conv2 = tfa.layers.InstanceNormalization()(conv2)
    conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
    conv2 = keras.layers.Dropout(rate=0.2)(conv2)
    conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
    # conv block -3
    conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv2)
    conv3 = keras.layers.GroupNormalization()(conv3)
    # conv3 = tfa.layers.InstanceNormalization()(conv3)
    conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
    conv3 = keras.layers.Dropout(rate=0.2)(conv3)
    # split for attention
    attention_data = keras.layers.Lambda(lambda x: x[:,:,:256])(conv3)
    attention_softmax = keras.layers.Lambda(lambda x: x[:,:,256:])(conv3)
    # attention mechanism
    attention_softmax = keras.layers.Softmax()(attention_softmax)
    multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
    # last layer
    dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
    dense_layer = keras.layers.GroupNormalization()(dense_layer)
    # dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
    # output layer
    flatten_layer = keras.layers.Flatten()(dense_layer)
    out = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

    return x,out

In [6]:
def dtw_classic(x, y):
    return dtw(x, y, method='classic')
def dtw_sakoechiba(x, y, window_size):
    return dtw(x, y, method='sakoechiba', options={'window_size': window_size})
def dtw_itakura(x, y, max_slope):
    return dtw(x, y, method='itakura', options={'max_slope': max_slope})

In [7]:
def cal_similarity(view1,view2,metric='boss'):
    similarity_list = []
    for i in range(view1.shape[0]):
        if metric == 'boss':
            similarity_list.append(boss(np.squeeze(view1[i]),np.squeeze(view2[i])))
        elif metric == 'dtw_classic':
            similarity_list.append(dtw_classic(np.squeeze(view1[i]),np.squeeze(view2[i])))
        elif metric == 'dtw_sakoechiba':
            similarity_list.append(dtw_sakoechiba(np.squeeze(view1[i]),np.squeeze(view2[i]),window_size=0.5))
        elif metric == 'dtw_itakura':
            similarity_list.append(dtw_itakura(np.squeeze(view1[i]),np.squeeze(view2[i]), max_slope=1.5))
        # elif metric == 'dtw_multiscale':
        #     similarity_list.append(dtw_multiscale(np.squeeze(view1[i]),np.squeeze(view2[i]), resolution=2) )
        # elif metric == 'dtw_fast':
        #     similarity_list.append(dtw_fast(np.squeeze(view1[i]),np.squeeze(view2[i]),radius = 1))
        else:
            print('other metric not implement yet.')
    return np.array(similarity_list)

# No Transfer Learning

In [ ]:
source_idx=0

lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,           # reduce by half
    patience=5,           # wait 5 epochs with no improvement
    min_lr=1e-5,          # don't go below this
    verbose=1
)

build_models = [('lstm', build_lstm), ('encoder', build_encoder), ('resnet', build_resnet)]


for build_model in build_models:
  model_name, build_fn = build_model

  for i in range(int(data_train.shape[2])):
    view_name = 'view' + str(i)
    # locals()[view_name+'_train'] = data_train[:,:,i].reshape(data_train.shape[0],data_train.shape[1],1)
    locals()[view_name+'_train'] = data_train_norm[:,:,i].reshape(data_train.shape[0],125,9)

    # locals()[view_name+'_test'] = data_test[:,:,i].reshape(data_test.shape[0],data_test.shape[1],1)
    locals()[view_name+'_test'] = data_test_norm[:,:,i].reshape(data_test.shape[0],125,9)

  print(f'********************* No transfer learning: {model_name} ********************')

  x, y = None, None
  if model_name == 'resnet':
    x, y = build_fn(locals()["view0_train"].shape[1:], 32, NUM_CLASSES)
  else:
    x, y = build_fn(locals()["view0_train"].shape[1:], NUM_CLASSES)


  model = keras.models.Model(inputs=x, outputs=y)
  adam = Adam(learning_rate=0.005)
  # chk = ModelCheckpoint('best_model.pkl', monitor=tf.keras.metrics.Accuracy(), save_best_only=True, mode='max', verbose=1)
  loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.25)
  # model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])

  history=model.fit(locals()['view'+str(source_idx)+'_train'],
              to_categorical(target_train, num_classes=NUM_CLASSES),
              epochs=50,
              batch_size=16,
              validation_data=(locals()['view'+str(source_idx)+'_test'],to_categorical(target_test, num_classes=NUM_CLASSES)),
              callbacks=lr_schedule,
              verbose=0)


  print(history.history['val_accuracy'][-1])

  model.save(f'no_transfer_model_{model_name}.h5')
  model.save(f'no_transfer_model_{model_name}.keras')
  with open(f'no_transfer_history_{model_name}.pkl', 'wb') as f:
      pickle.dump(history, f)

# Naive Transfer Learning

In [ ]:
source_idx=0

build_models = [('lstm', build_lstm), ('encoder', build_encoder), ('resnet', build_resnet)]


for model_name, build_fn in build_models:
  for i in range(int(data_train.shape[2])):
    view_name = 'view' + str(i)
    # locals()[view_name+'_train'] = data_train[:,:,i].reshape(data_train.shape[0],data_train.shape[1],1)
    locals()[view_name+'_train'] = data_train_norm[:,:,i].reshape(data_train.shape[0],125,9)

    # locals()[view_name+'_test'] = data_test[:,:,i].reshape(data_test.shape[0],data_test.shape[1],1)
    locals()[view_name+'_test'] = data_test_norm[:,:,i].reshape(data_test.shape[0],125,9)

  print(f'********************* Naive transfer learning {model_name} ********************')
  x, y = None, None
  if model_name == 'resnet':
    x, y = build_fn(locals()["view0_train"].shape[1:], 32, NUM_CLASSES)
  else:
    x, y = build_fn(locals()["view0_train"].shape[1:], NUM_CLASSES)

  model = keras.models.Model(inputs=x, outputs=y)
  adam = Adam(learning_rate=0.005)
  loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.25)
  # model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])



  # transfer learning
  for i in range(int(data_train.shape[2])):
    print(f'Source idx: {i}')
    if i != source_idx:
        view_name = 'view' + str(i)
        weight_name = 'weight' + str(source_idx) + str(i)
        model.fit(locals()[view_name+'_train'],
                  to_categorical(target_train),
                  epochs=30,
                  batch_size=16,
                  validation_data=(locals()[view_name+'_test'],to_categorical(target_test)),
                  verbose=0)
  # on target domain
  history=model.fit(locals()['view'+str(source_idx)+'_train'],
              to_categorical(target_train),
              epochs=40,
              batch_size=16,
              validation_data=(locals()['view'+str(source_idx)+'_test'], to_categorical(target_test)),
              verbose=0)

  print(history.history['val_accuracy'][-1])

  model.save(f'naive_transfer_model_{model_name}.h5')
  model.save(f'naive_transfer_model_{model_name}.keras')
  with open(f'naive_transfer_history_{model_name}.pkl', 'wb') as f:
      pickle.dump(history, f)

# Weighted Transfer Learning

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

source_idx=0

build_models = [('lstm', build_lstm), ('encoder', build_encoder), ('resnet', build_resnet)]

# for metric in ['boss','dtw_classic','dtw_sakoechiba','dtw_itakura']:
for metric in ['dtw_classic']:
  mode_name=metric+'+kde'
  for i in range(int(data_train.shape[2])):
      view_name = 'view' + str(i)
      locals()[view_name+'_train'] = data_train_norm[:,:,i].reshape(data_train.shape[0],data_train.shape[1],1)
      locals()[view_name+'_test'] = data_test_norm[:,:,i].reshape(data_test.shape[0],data_test.shape[1],1)

  for i in range(int(data_train.shape[2])):
          similarity_name = 'similarity' + str(source_idx)
          print(i)
          if i != source_idx:
              locals()[similarity_name+str(i)] = cal_similarity(locals()['view'+str(source_idx)+'_train'],locals()['view'+str(i)+'_train'],metric)
      # # KDE
  for i in range(int(data_train.shape[2])):
          kde_name = 'kde'+ str(source_idx)
          similarity_name = 'similarity' + str(source_idx)
          if i != source_idx:
              locals()[kde_name+str(i)] = KernelDensity(kernel='gaussian', bandwidth=7.8).fit(locals()[similarity_name+str(i)].reshape(locals()[similarity_name+str(i)].flatten().shape[0],1))
  # weight
  weight_all = 0
  for i in range(int(data_train.shape[2])):
          if i != source_idx:
              kde_name = 'kde'+ str(source_idx) + str(i)
              weight_name = 'weight' + str(source_idx) + str(i)
              locals()[weight_name] =  np.mean(locals()[kde_name].sample(10,random_state=0),axis=0)[0]
              weight_all += locals()[weight_name]
  for i in range(int(data_train.shape[2])):
    view_name = 'view' + str(i)
    # locals()[view_name+'_train'] = data_train[:,:,i].reshape(data_train.shape[0],data_train.shape[1],1)
    locals()[view_name+'_train'] = data_train_norm[:,:,i].reshape(data_train.shape[0],125,9)

    # locals()[view_name+'_test'] = data_test[:,:,i].reshape(data_test.shape[0],data_test.shape[1],1)
    locals()[view_name+'_test'] = data_test_norm[:,:,i].reshape(data_test.shape[0],125,9)


  for model_name, build_fn in build_models:

      print(f'********************* Weighted transfer learning {model_name} ********************')

      x, y = None, None
      if model_name == 'resnet':
        x, y = build_fn(locals()["view0_train"].shape[1:], 32, NUM_CLASSES)
      else:
        x, y = build_fn(locals()["view0_train"].shape[1:], NUM_CLASSES)

      model = keras.models.Model(inputs=x, outputs=y)
      adam = Adam(learning_rate=0.0005)
      loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.25)
      # model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
      model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])

      # transfer learning
      for i in range(int(data_train.shape[2])):
              if i != source_idx:
                  view_name = 'view' + str(i)
                  weight_name = 'weight' + str(source_idx) +str(i)
                  model.fit(locals()[view_name+'_train'], to_categorical(target_train), epochs=int(30*7*locals()[weight_name]/weight_all)+1,batch_size=16,validation_data=(locals()[view_name+'_test'],to_categorical(target_test)),verbose=1)
      # on target domain
      history=model.fit(locals()['view'+str(source_idx)+'_train'],
                  to_categorical(target_train),
                  epochs=40,
                  batch_size=16,
                  validation_data=(locals()['view'+str(source_idx)+'_test'],to_categorical(target_test)),verbose=1)

      model.save(f'weighted_transfer_model_{model_name}.h5')
      model.save(f'weighted_transfer_model_{model_name}.keras')
      with open(f'weighted_transfer_history_{model_name}.pkl', 'wb') as f:
          pickle.dump(history, f)